In [15]:
!rm -rf camp_data
!git clone https://github.com/HongJeSeong/camp_data.git

Cloning into 'camp_data'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 86 (delta 32), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (86/86), done.


In [16]:
!ls camp_data

covidKorea.csv	img		    README.md	src
covidTime.csv	influenzaCovid.csv  review.txt	wine.csv


In [108]:
import pandas as pd
df = pd.read_csv("camp_data/review.txt", names=['sentence', 'label'], sep='\t')
df

,sentence,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
2743,I think food should have flavor and texture an...,0
2744,Appetite instantly gone.,0
2745,Overall I was not impressed and would not go b...,0
2746,"The whole experience was underwhelming, and I ...",0


In [123]:
from sklearn.model_selection import train_test_split
x = df['sentence'].values
y = df['label'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20)

데이터를 읽어 들이고
단어 빈도에 따른 사용할 단어 개수의 최대값. 가장 빈번하게 사용되는 num_words개의 단어만 저장

In [127]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test) 
vocab_size=len(tokenizer.word_index)+1

In [125]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'i': 3,
 'a': 4,
 'is': 5,
 'it': 6,
 'to': 7,
 'this': 8,
 'of': 9,
 'was': 10,
 'in': 11,
 'for': 12,
 'not': 13,
 'that': 14,
 'with': 15,
 'my': 16,
 'very': 17,
 'good': 18,
 'on': 19,
 'great': 20,
 'you': 21,
 'but': 22,
 'have': 23,
 'movie': 24,
 'are': 25,
 'as': 26,
 'so': 27,
 'phone': 28,
 'film': 29,
 'be': 30,
 'all': 31,
 'one': 32,
 '0': 33,
 'had': 34,
 '1': 35,
 'at': 36,
 'food': 37,
 'like': 38,
 'just': 39,
 'place': 40,
 "it's": 41,
 'time': 42,
 'service': 43,
 'an': 44,
 'were': 45,
 'if': 46,
 'from': 47,
 'bad': 48,
 'really': 49,
 'there': 50,
 'they': 51,
 'we': 52,
 'well': 53,
 'out': 54,
 'has': 55,
 'would': 56,
 'about': 57,
 'no': 58,
 'or': 59,
 'your': 60,
 'only': 61,
 'by': 62,
 'best': 63,
 "don't": 64,
 'even': 65,
 'here': 66,
 'ever': 67,
 'up': 68,
 'also': 69,
 'will': 70,
 'back': 71,
 'me': 72,
 'when': 73,
 'more': 74,
 'than': 75,
 'quality': 76,
 'go': 77,
 'what': 78,
 'love': 79,
 'he': 80,
 "i've": 81,
 'can': 

In [126]:
x_train[1], xtrain[1]

('The headset fulfills my requirements so I am happy with my purchase.', [])

![padding](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/padding.PNG)

In [114]:
xtrain=pad_sequences(xtrain,padding='post', maxlen=10)
xtest=pad_sequences(xtest,padding='post', maxlen=10)

In [115]:
xtrain[1]

array([ 1, 16, 27,  3, 15, 16,  0,  0,  0,  0], dtype=int32)

In [119]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
embedding_dim=50
model=Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=30))
model.add(LSTM(units=50,return_sequences=True))
model.add(LSTM(units=10))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

![d](https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
(https://raw.githubusercontent.com/HongJeSeong/camp_data/main/img/dropout.PNG)
학습의 오버피팅을 줄이기 위한 레이어

In [120]:
history = model.fit(xtrain,y_train, epochs=20, batch_size=16, verbose=0)

loss, acc = model.evaluate(xtrain, y_train, verbose=False)
print("Train Accuarcy: ",acc)
loss, acc = model.evaluate(xtest, y_test, verbose=False)
print("Test Accuracy: ", acc)

Train Accuarcy:  0.7593266367912292
Test Accuracy:  0.6418181657791138
